In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from config import username
from config import password

COVID_19_Activity_df = pd.read_csv('https://query.data.world/s/ynb7kpmvt5dw26vogf5ozoqjqpyhb2')
COVID_19_Activity_df

In [ ]:
#Drop the rows where at least one element is missing
COVID_19_Activity_reduced_df = COVID_19_Activity_df.dropna()
COVID_19_Activity_reduced_df

In [ ]:
#Identify incomplete rows
COVID_19_Activity_reduced_df["PROVINCE_STATE_NAME"].value_counts()

In [ ]:
#Groupby State
State_COVID_19_Activity_df = COVID_19_Activity_reduced_df.groupby(["PROVINCE_STATE_NAME"]).sum()
State_COVID_19_Activity_df = State_COVID_19_Activity_df.reset_index()
State_COVID_19_Activity_df

In [ ]:
# State_COVID_19_Activity_df.drop("COUNTY_FIPS_NUMBER", axis=1, inplace=True)
State_COVID_19_Activity_df.drop(["COUNTY_FIPS_NUMBER", "PEOPLE_POSITIVE_CASES_COUNT","PEOPLE_DEATH_COUNT"], axis=1, inplace=True)
State_COVID_19_Activity_df

In [ ]:
#Read and format US population dataset
US_Population_df = pd.read_csv('Datasets/NST-EST2021-POP.csv')
US_Population_df = US_Population_df.rename(columns = {'STATE': 'PROVINCE_STATE_NAME'})
US_Population_df['PROVINCE_STATE_NAME'] = US_Population_df['PROVINCE_STATE_NAME'].apply(lambda x: x.replace(".",""))
US_Population_df['POPULATION'] = US_Population_df['POPULATION'].apply(lambda x: x.replace(",",""))
US_Population_df

In [ ]:
#Merge and format data
population_covid_df = pd.merge(US_Population_df, State_COVID_19_Activity_df, on=['PROVINCE_STATE_NAME'])
population_covid_df["POPULATION"]=population_covid_df["POPULATION"].astype(int)
#Population_COVID_df.dtypes
population_covid_df

In [ ]:
#Add %Cases and %Death count with US state % population
population_covid_df["Perc_Cases"]=(population_covid_df["PEOPLE_POSITIVE_NEW_CASES_COUNT"]/population_covid_df["POPULATION"]*100)
population_covid_df["Perc_Death"]=(population_covid_df["PEOPLE_DEATH_NEW_COUNT"]/population_covid_df["POPULATION"]*100)
population_covid_df=population_covid_df.sort_values("Perc_Cases")
population_covid_df

In [ ]:
population_covid_df.columns

In [ ]:
population_covid_df.columns=['province_state_name', 'population',
       'people_death_new_count', 'people_positive_new_cases_count',
       'perc_cases', 'perc_death']
population_covid_df

In [ ]:
#Connect to local database
protocol = 'postgresql'
host = 'localhost'
port = 5432
database_name = 'population_covid_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
#Confirm table
engine.table_names()

In [ ]:
#Load final dataframe to database
population_covid_df.to_sql(name='state_covid_19_activity', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from state_covid_19_activity', con=engine).head()